# Quantum Fourier Transform

The classical Fourier transform is an important tool in wave and signal analysis that breaks a function into components, each with a different frequency.
Its discrete analogue, the Discrete Fourier transform acts on $n$ complex numbers $x_0,\ldots,x_{N-1}$ and transforms it to another sequence of $n$ complex numbers $\tilde x_0,\ldots,\tilde x_{N-1}$ via:

$$\tilde x_k = \sum_{y=0}^{N-1}e^{-\frac{2\pi ikn}N} \cdot x_k$$

The quantum Fourier transform (commonly abbreviated as QFT) on $n$ qubits (with $N=2^n$), achieves an analogous effect, and is given by the following equation (for every basis state $x\in\{0,1\}^n$):

$$\text{QFT}(\lvert x\rangle) = \frac 1{\sqrt N}\sum_{y=0}^{N-1}e^{\frac{2\pi ixy}N}\lvert y\rangle\qquad \qquad (1)$$

Here, we abuse notation and associate a bitstring $x\in\{0,1\}^n$ to the integer $\sum_{j=0}^{n-1}2^{n-1-j}x_j$.

Effectively, one can think of QFT as a change of basis from the computational basis to the "Fourier basis".


## Factorization of QFT

It turns out (after some algebraic manipulations) that one is able to factorize the sum in Eq. $(1)$ into a product of pure states

$$\text{QFT}(\lvert x\rangle) = \frac 1{\sqrt N}\left(\vert 0\rangle + e^{\frac{2\pi ix}{2^1}}\vert 1\rangle\right)\otimes \left(\vert 0\rangle + e^{\frac{2\pi ix}{2^2}}\vert 1\rangle\right)\otimes \cdots \otimes \left(\vert 0\rangle + e^{\frac{2\pi ix}{2^n}}\vert 1\rangle\right).$$

This formulation tells us exactly how to implement a quantum circuit achieving QFT.

## Controlled Phase Rotations
Before looking at the circuit structure though, we define the gate
$$R_k = \begin{pmatrix}1 & 0\\ 0 & e^{\frac{2\pi i}{2^k}}\end{pmatrix},$$
and recall that a controlled version of this gate will change the phase of the target qubit depending on the value of the control qubit.
In other words, $CR_k(\lvert 0\rangle|x\rangle) = |0\rangle \lvert x\rangle$ and 

\begin{aligned}CR_k(\lvert 1\rangle \lvert x\rangle) = \lvert 1\rangle\otimes R_k(\lvert x \rangle) &=\lvert 1\rangle\otimes R_k\left( \langle 0\lvert x\rangle \lvert  0\rangle + \langle 1 \lvert x\rangle \lvert 1\rangle\right)
 \\ & = \lvert 1\rangle\otimes \left( \langle 0 \lvert x\rangle \lvert  0\rangle + \langle 1 \lvert x\rangle e^{\frac{2\pi i}{2^k}} \lvert 1\rangle\right)\end{aligned}

More compactly,
$$CR_k(\lvert y\rangle \lvert x\rangle) = \lvert y\rangle \otimes \left( \langle 0 \vert x\rangle \lvert  0\rangle + \langle 1 \vert x\rangle e^{\frac{2\pi iy}{2^k}} \lvert 1\rangle\right).$$

Therefore, $$CR_k = \begin{pmatrix}1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0\\ 0& 0& 1 & 0\\ 0 & 0&0 &e^{\frac{2\pi i}{2^k}}\end{pmatrix} = \text{CPhase}\left(\frac{2\pi}{2^k}\right).$$

## Circuit Structure

Now we have all the ingredients required to construct a circuit achieving QFT.
Below is an example circuit on $n=4$ qubits:

![Circuit Image](img/121_qft_ckt.png)

We shall work our way through the states $\lvert \varphi_1\rangle$ through $\lvert \varphi_4\rangle$.


In $\lvert \varphi_1\rangle$, let us analyze the state of the first qubit on the basis states $x_0\in\{0,1\}$. Then $\lvert x_0\rangle$ is mapped to (let us ignore normalizations for simplifying calculations):
+ $H$: $\lvert 0\rangle + e^{\pi i x_0}\lvert 1\rangle$,
+ $R_2$: $\lvert 0\rangle + e^{\pi i x_0 + \frac{2\pi i x_1}{2^2}}\lvert 1\rangle$
+ $R_3$: $\lvert 0\rangle + e^{\pi i x_0 + \frac{2\pi i x_1}{2^2} + \frac{2\pi i x_2}{2^3}}\lvert 1\rangle$
+ $R_4$: $\lvert 0\rangle + e^{\pi i x_0 + \frac{2\pi i x_1}{2^2} + \frac{2\pi i x_2}{2^3} + \frac{2\pi i x_3}{2^4}}\lvert 1\rangle = \lvert 0\rangle + e^{\frac{2\pi i}{2^4} (8x_0 + 4x_1 + 2x_2 + x_3)}\lvert 1\rangle = |0\rangle + e^{\frac{2\pi i x}{2^4}}\lvert 1\rangle$.

Hence, $\lvert \varphi_1\rangle = \left(\lvert 0\rangle + e^{\frac{2\pi i x}{2^4}}\lvert 1\rangle\right)\otimes \lvert x_1x_2x_3\rangle$.

It is now easy to see via an inductive argument that $\lvert \varphi_2\rangle$ changes qubit $\lvert x_1\rangle$ to:
+ $\lvert 0\rangle + e^{\frac{2\pi i}{2^3}(4x_1 + 2x_2 + x_3)}\lvert 1\rangle$.  

However, as $e^{2\pi ix_0} = 1$, we have $e^{\frac{2\pi i}{2^3}(4x_1 + 2x_2 + x_3)}=e^{\frac{2\pi i}{2^3}(8x_0 + 4x_1 + 2x_2 + x_3)}= e^{\frac{2\pi ix}{2^3}}$.

So, $\lvert \varphi_2\rangle = \left(\lvert 0\rangle + e^{\frac{2\pi i x}{2^4}}\lvert 1\rangle\right)\otimes \left(\lvert 0\rangle + e^{\frac{2\pi i x}{2^3}}\lvert 1\rangle\right)\otimes \lvert x_2x_3\rangle$.

Continuing, we can check that $\vert \varphi_4 \rangle$ gives us the reverse order of the qubits that we wanted, and hence we do a double swap to complete the circuit schematic of QFT.

## Writing QFT in Blueqat-SDK

In [1]:
# Install Blueqat-SDK if you haven't already!
# !pip install blueqat

In [2]:
# Import Modules
from blueqat import Circuit
import math

In [3]:
# Function to apply qft on a list of qubits of circuit
def apply_qft(circuit: Circuit(), qubits):
    num_qubits = len(qubits)
    for i in range(num_qubits):
        circuit.h[qubits[i]]
        for j in range(i+1, num_qubits):
            circuit.cphase(math.pi/(2 ** (j-i)))[qubits[j],qubits[i]] # Apply gate CR_{j-i}(qubit j, qubit i)
    # Reverse the order of qubits at the end
    for i in range(int(num_qubits/2)):
        circuit.swap(qubits[i],qubits[num_qubits-i-1])

### Testing on a circuit

In [4]:
n = 4  # number of qubits to apply QFT on
qc = Circuit()
qc.x[:]  # Prepare the state |1111>
apply_qft(qc, range(n))
qc.run()

array([ 2.50000000e-01+0.j        ,  2.30969883e-01-0.09567086j,
        1.76776695e-01-0.1767767j ,  9.56708581e-02-0.23096988j,
       -1.53080850e-17-0.25j      , -9.56708581e-02-0.23096988j,
       -1.76776695e-01-0.1767767j , -2.30969883e-01-0.09567086j,
       -2.50000000e-01+0.j        , -2.30969883e-01+0.09567086j,
       -1.76776695e-01+0.1767767j , -9.56708581e-02+0.23096988j,
        1.53080850e-17+0.25j      ,  9.56708581e-02+0.23096988j,
        1.76776695e-01+0.1767767j ,  2.30969883e-01+0.09567086j])

The QFT only changes phases of individual qubits, and hence cannot be physically observed via measurements. Below is a visual representation of the above statevector obtained from $\text{QFT}(|1111\rangle)$:

![Statevector](img/121_qft_statevec.png)


## Applications of QFT

QFT has wide applications in Algorithms design, and is the building block for many influential quantum algorithms such as Quantum Phase Estimation [[1]] , Shor's Algorithm [[2]], and the Hidden Subgroup Problem [[3]].

[1]: https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm
[2]: https://en.wikipedia.org/wiki/Shor's_algorithm
[3]: https://en.wikipedia.org/wiki/Hidden_subgroup_problem